In [25]:
! pip -q install pandas plotly
! pip -q install nbformat
! pip -q install kaleido==0.2.1


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [26]:
import pandas as pd
import plotly.express as px
import kaleido
import plotly.graph_objects as go
import plotly.colors as pc
import glob
import json
from plotly.subplots import make_subplots
import math
import os
import numpy as np
import plotly.io as pio
pio.kaleido.scope.mathjax = None

In [27]:
! plotly_get_chrome

/bin/bash: plotly_get_chrome: command not found


# Reading the results

In [28]:
result_files = glob.glob("eval_results/fineweb/*/*_iter_*/*.json")
#baseline_hptl2_dedup = glob.glob("/scratch/project_462000353/pyysalos/second-hplt-eval/eval_results/results/hplt_v2_dedup_iter_00*/*.json")
#baseline_fineweb = glob.glob("/scratch/project_462000353/pyysalos/second-hplt-eval/eval_results/results/fineweb_iter_00*/*.json")

tasks = {"custom|arc:easy|0":"arc_easy",
         "custom|arc:challenge|0":"arc_challenge",
         "custom|hellaswag|0":"hellaswag",
         "custom|commonsense_qa|0": "commonsense_qa",
         "custom|openbookqa|0": "openbookqa",
         "custom|piqa|0": "piqa",
         "custom|siqa|0": "siqa",
         "custom|mmlu:_average|0": "mmlu",
         "custom|winogrande|0": "winogrande",
}



In [29]:
def extract_results(result_files, recalc_mean=False):
    data = []
    for f in result_files:
        splitted = f.split("/")
        #print(splitted)
        register = splitted[-3] #splitted[-2].split("_iter_")[0]
        step = splitted[-2].split("_")[-1] #splitted[-2].replace(f'{register}_iter_', "")
        step=int(step)
        #print(register)
        #print(step)
        with open(f, 'r') as result:
            j = json.load(result)
            results = {"step":int(step)}
            if step > 50000:
                continue
            results_all = j["results"]["all"]
            results = {**results, **results_all}
            for task, task_name in tasks.items():
                r = {k+f"_{task_name}":v for k,v in j["results"][task].items()}
                results = {**results, **r}
            #print(results)
            df = pd.DataFrame.from_dict({register:{"step":int(step), **results}})
            data.append(df.T)
            #print(register, j["results"]["all"])
            #data[register] = {step: j["results"]["all"]}

    df = pd.concat(data)
    df = df.reset_index()
    df = df.rename(columns={"index":"register"})
    # problem: step does not go to correct place on the plot
    # sort it as numerical, then make it categorical so plotting works
    # ordered=True does nothing here because plotly does not respect it; only sorting works
    df=df.sort_values("step")
    df["tokens"] = df["step"]*2.1e6
    #df["step"] = pd.Categorical(df['step'], categories=sorted(df['step'].astype(int).unique()), ordered=True)
    df["step"]= df['step'].apply(lambda x: str(round(x)))
    #df["tokens"]= df['tokens'].apply(lambda x: millify(x))
    if recalc_mean:
        df["acc_norm_original"] = df["acc_norm"]
        #df["acc_norm"] = df[["acc_norm_hellaswag", "acc_norm_openbookqa", "acc_norm_piqa", "acc_norm_arc_easy"]].mean(axis=1)
        df["acc_norm"] = df[[f"acc_norm_{taskname}" for taskname in tasks.values()]].mean(axis=1)
        df["stderr_norm"] = df[[f"acc_norm_stderr_{taskname}" for taskname in tasks.values()]].apply(lambda x: x**2).sum(axis=1)
        df["stderr_norm"] = df["stderr_norm"].apply(lambda x: math.sqrt(x))
    return df

df1 = extract_results(result_files, recalc_mean=True)

In [30]:
df = df1.sort_values(["register", "tokens"]).reset_index()
display(df)
print(df.columns)

display(df[df.register=="IN"])


,index,register,acc,acc_arc_challenge,acc_arc_easy,acc_commonsense_qa,acc_hellaswag,acc_mmlu,acc_norm,acc_norm_arc_challenge,...,acc_stderr_mmlu,acc_stderr_openbookqa,acc_stderr_piqa,acc_stderr_siqa,acc_stderr_winogrande,acc_winogrande,step,tokens,acc_norm_original,stderr_norm
0,279,HI,0.245950,0.171,0.307,0.252,0.296,0.233294,0.351006,0.222,...,0.031361,0.016158,0.015071,0.015196,0.015819,0.496,1000,2.100000e+09,0.264036,0.054150
1,265,HI,0.252454,0.182,0.369,0.265,0.347,0.236395,0.377155,0.208,...,0.031445,0.016244,0.014633,0.015460,0.015787,0.532,2000,4.200000e+09,0.273118,0.054744
2,259,HI,0.257911,0.194,0.378,0.306,0.380,0.241425,0.388144,0.235,...,0.031593,0.016244,0.014356,0.015275,0.015817,0.509,3000,6.300000e+09,0.274549,0.054673
3,281,HI,0.258030,0.177,0.399,0.294,0.380,0.240596,0.400953,0.220,...,0.031554,0.016894,0.014013,0.015333,0.015796,0.527,4000,8.400000e+09,0.277428,0.055073
4,272,HI,0.260565,0.199,0.407,0.301,0.397,0.242293,0.406407,0.242,...,0.031707,0.016328,0.013825,0.015325,0.015755,0.545,5000,1.050000e+10,0.276535,0.054875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,140,ne,0.284349,0.233,0.482,0.340,0.374,0.267398,0.411706,0.267,...,0.032736,0.018170,0.015130,0.015453,0.015685,0.565,46000,9.660000e+10,0.301991,0.056417
950,102,ne,0.283513,0.243,0.496,0.332,0.380,0.266409,0.414428,0.265,...,0.032717,0.017491,0.015241,0.015512,0.015672,0.568,47000,9.870000e+10,0.303657,0.056491
951,115,ne,0.287017,0.235,0.480,0.331,0.379,0.270230,0.414604,0.270,...,0.032892,0.018039,0.015081,0.015473,0.015631,0.577,48000,1.008000e+11,0.303319,0.056481
952,124,ne,0.284679,0.241,0.482,0.335,0.382,0.267915,0.416693,0.269,...,0.032804,0.017771,0.015177,0.015453,0.015693,0.563,49000,1.029000e+11,0.303435,0.056557


Index(['index', 'register', 'acc', 'acc_arc_challenge', 'acc_arc_easy',
       'acc_commonsense_qa', 'acc_hellaswag', 'acc_mmlu', 'acc_norm',
       'acc_norm_arc_challenge', 'acc_norm_arc_easy',
       'acc_norm_commonsense_qa', 'acc_norm_hellaswag', 'acc_norm_mmlu',
       'acc_norm_openbookqa', 'acc_norm_piqa', 'acc_norm_siqa',
       'acc_norm_stderr', 'acc_norm_stderr_arc_challenge',
       'acc_norm_stderr_arc_easy', 'acc_norm_stderr_commonsense_qa',
       'acc_norm_stderr_hellaswag', 'acc_norm_stderr_mmlu',
       'acc_norm_stderr_openbookqa', 'acc_norm_stderr_piqa',
       'acc_norm_stderr_siqa', 'acc_norm_stderr_winogrande',
       'acc_norm_winogrande', 'acc_openbookqa', 'acc_piqa', 'acc_siqa',
       'acc_stderr', 'acc_stderr_arc_challenge', 'acc_stderr_arc_easy',
       'acc_stderr_commonsense_qa', 'acc_stderr_hellaswag', 'acc_stderr_mmlu',
       'acc_stderr_openbookqa', 'acc_stderr_piqa', 'acc_stderr_siqa',
       'acc_stderr_winogrande', 'acc_winogrande', 'step', 'token

,index,register,acc,acc_arc_challenge,acc_arc_easy,acc_commonsense_qa,acc_hellaswag,acc_mmlu,acc_norm,acc_norm_arc_challenge,...,acc_stderr_mmlu,acc_stderr_openbookqa,acc_stderr_piqa,acc_stderr_siqa,acc_stderr_winogrande,acc_winogrande,step,tokens,acc_norm_original,stderr_norm
405,604,IN,0.255274,0.161,0.377,0.231,0.281,0.244611,0.344821,0.209,...,0.031830,0.015807,0.015654,0.015091,0.015787,0.532,1000,2.100000e+09,0.270357,0.054767
406,619,IN,0.268975,0.196,0.437,0.269,0.305,0.257200,0.367394,0.238,...,0.032384,0.015717,0.015600,0.015309,0.015812,0.515,2000,4.200000e+09,0.284817,0.055535
407,600,IN,0.275709,0.204,0.479,0.283,0.324,0.263703,0.379695,0.257,...,0.032648,0.015440,0.015561,0.015101,0.015805,0.521,3000,6.300000e+09,0.292302,0.055902
408,621,IN,0.278320,0.210,0.476,0.284,0.331,0.265593,0.388786,0.253,...,0.032748,0.016072,0.015480,0.015232,0.015789,0.531,4000,8.400000e+09,0.296850,0.056130
409,620,IN,0.278320,0.210,0.476,0.284,0.331,0.265593,0.388786,0.253,...,0.032748,0.016072,0.015480,0.015232,0.015789,0.531,4000,8.400000e+09,0.296850,0.056130
410,631,IN,0.279271,0.219,0.492,0.309,0.340,0.265011,0.398560,0.251,...,0.032551,0.016328,0.015365,0.015275,0.015769,0.540,5000,1.050000e+10,0.301617,0.056313
411,599,IN,0.288370,0.233,0.521,0.309,0.342,0.274299,0.401605,0.253,...,0.033144,0.016816,0.015325,0.015325,0.015783,0.534,6000,1.260000e+10,0.306699,0.056503
412,630,IN,0.291216,0.243,0.516,0.323,0.343,0.277896,0.407192,0.257,...,0.033080,0.016657,0.015325,0.015196,0.015814,0.513,7000,1.470000e+10,0.309437,0.056540
413,632,IN,0.292845,0.239,0.544,0.332,0.345,0.278368,0.410241,0.275,...,0.033142,0.016737,0.015258,0.015388,0.015801,0.524,8000,1.680000e+10,0.311963,0.056580
414,625,IN,0.291391,0.234,0.547,0.339,0.364,0.276621,0.413378,0.263,...,0.033058,0.016412,0.015300,0.015317,0.015796,0.527,9000,1.890000e+10,0.316048,0.056750


## Rolling average to make the line plots more readable

In [31]:
# rolling average over how many steps?

unique_registers = np.unique(df["register"])
print(unique_registers)
n = 3

def rolling_average(df, register, n=5):
    df_p = df[df.register==register].rolling(n, center=True, axis=0).mean()
    return df_p


dfs_rolled = []
for register in unique_registers:
    #print(register)
    p = rolling_average(df, register, n=n)
    p["register"]=register
    dfs_rolled.append(p)

df_rolled = pd.concat(dfs_rolled, axis=0).dropna()

display(df_rolled)
#display(df_rolled[df_rolled.register=="HI-IN"])




['HI' 'HI-IN' 'HI-IN_HI_dtp' 'HI-IN_HI_dtp_NA_OP' 'HI-IN_HI_dtp_NA_OP_ID'
 'HI-IN_HI_dtp_NA_OP_ID_SP' 'HI-IN_HI_dtp_OP' 'ID' 'IN' 'IP' 'LY' 'MT'
 'NA' 'OP' 'SP' 'dtp' 'fineweb' 'hplt-v2-dedup' 'ne']


/tmp/ipykernel_193016/984316175.py:8: FutureWarning:

Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['register'], dtype='object')

/tmp/ipykernel_193016/984316175.py:8: FutureWarning:

Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['register'], dtype='object')

/tmp/ipykernel_193016/984316175.py:8: FutureWarning:

Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['register'], dtype='object')

/tmp/ipykernel_193016/984316175.py:8: FutureWarning:

Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise T

,index,acc,acc_arc_challenge,acc_arc_easy,acc_commonsense_qa,acc_hellaswag,acc_mmlu,acc_norm,acc_norm_arc_challenge,acc_norm_arc_easy,...,acc_stderr_openbookqa,acc_stderr_piqa,acc_stderr_siqa,acc_stderr_winogrande,acc_winogrande,step,tokens,acc_norm_original,stderr_norm,register
1,267.666667,0.252105,0.182333,0.351333,0.274333,0.341000,0.237038,0.372102,0.221667,0.352333,...,0.016215,0.014687,0.015310,0.015807,0.512333,2000.0,4.200000e+09,0.270567,0.054522,HI
2,268.333333,0.256132,0.184333,0.382000,0.288333,0.369000,0.239472,0.388751,0.221000,0.381667,...,0.016460,0.014334,0.015356,0.015800,0.522667,3000.0,6.300000e+09,0.275032,0.054830,HI
3,270.666667,0.258835,0.190000,0.394667,0.300333,0.385667,0.241438,0.398501,0.232333,0.396667,...,0.016489,0.014065,0.015311,0.015789,0.527000,4000.0,8.400000e+09,0.276171,0.054873,HI
4,278.333333,0.260395,0.190000,0.406333,0.303000,0.395667,0.242415,0.405845,0.234333,0.403667,...,0.016626,0.013951,0.015338,0.015780,0.534333,5000.0,1.050000e+10,0.277846,0.055017,HI
5,272.000000,0.261869,0.201667,0.415667,0.314667,0.407667,0.243178,0.408728,0.240333,0.408000,...,0.016652,0.013905,0.015385,0.015783,0.532333,6000.0,1.260000e+10,0.278198,0.054976,HI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,138.000000,0.284413,0.234000,0.490000,0.342667,0.371667,0.267593,0.413454,0.262667,0.478000,...,0.017391,0.015183,0.015488,0.015657,0.571000,45000.0,9.450000e+10,0.301135,0.056397,ne
949,128.333333,0.283880,0.236333,0.487667,0.340667,0.375667,0.266997,0.414122,0.264667,0.473667,...,0.017439,0.015189,0.015497,0.015679,0.566333,46000.0,9.660000e+10,0.301816,0.056448,ne
950,119.000000,0.284960,0.237000,0.486000,0.334333,0.377667,0.268013,0.413579,0.267333,0.470667,...,0.017900,0.015150,0.015480,0.015663,0.570000,47000.0,9.870000e+10,0.302989,0.056463,ne
951,113.666667,0.285070,0.239667,0.486000,0.332667,0.380333,0.268185,0.415242,0.268000,0.471333,...,0.017767,0.015166,0.015480,0.015665,0.569333,48000.0,1.008000e+11,0.303470,0.056510,ne


# Actual final plotting

Here I chose the colormaps, put dashed lines for epoch limits, etc. Using the rolled average as before since rugged lines are hard to follow by eye.


In [32]:
# constants and helpers

organized_registers=['HI-IN_HI_dtp','HI-IN_HI_dtp_OP','HI-IN_HI_dtp_NA_OP','HI-IN_HI_dtp_NA_OP_ID','HI-IN_HI_dtp_NA_OP_ID_SP','fineweb','HI-IN','hplt-v2-dedup','dtp','HI','OP','NA','IN','ID','SP','ne','IP','MT','LY']
organized_registers2=['HI-IN_HI_dtp_OP','HI-IN_HI_dtp_NA_OP','HI-IN_HI_dtp','HI-IN_HI_dtp_NA_OP_ID','HI-IN_HI_dtp_NA_OP_ID_SP','fineweb','HI-IN','hplt-v2-dedup','dtp','HI','OP','NA','IN','ID','SP','ne','IP','MT','LY']
vanilla_registers=['HI-IN','dtp','HI','OP','IN','NA','ID','SP','IP','ne','MT','LY']
baselines=["fineweb", "hplt-v2-dedup"]
combinations = ['HI-IN_HI_dtp','HI-IN_HI_dtp_OP','HI-IN_HI_dtp_NA_OP','HI-IN_HI_dtp_NA_OP_ID','HI-IN_HI_dtp_NA_OP_ID_SP']


register_name_map_old = {'HI-IN_HI_dtp_OP': "HI-IN-HI-dtp-OP",
                     'HI-IN_HI_dtp_NA_OP': "HI-IN-HI-dtp-OP-NA",
                     'HI-IN_HI_dtp': "HI-IN-HI-dtp",
                     'HI-IN_HI_dtp_NA_OP_ID':"HI-IN-HI-dtp-OP-NA-ID",
                     'HI-IN_HI_dtp_NA_OP_ID_SP': "HI-IN-HI-dtp-OP-NA-ID-SP",
                     'hplt-v2-dedup': "All Registers (full HPLT v2)",#"HPLT v2",
                     'fineweb': "FineWeb",
                     'HI-IN': "Instructive-Informational",
                     'HI': "How-to-Instructions",
                     'ID': "Interactive Discussion",
                     'IN': "Informational Description",
                     "IP": "Informational Persuasion",
                     "LY": "Lyrical",
                     "MT": "Machine Translation",
                     "NA": "Narrative",
                     "OP": "Opinion",
                     "SP": "Spoken",
                     "ne": "News",
                     "dtp": "Description"}
register_name_map = {'HI-IN_HI_dtp_OP': "HI-IN-HI-dtp-OP",
                     'HI-IN_HI_dtp_NA_OP': "HI-IN-HI-dtp-OP-NA",
                     'HI-IN_HI_dtp': "HI-IN-HI-dtp",
                     'HI-IN_HI_dtp_NA_OP_ID':"HI-IN-HI-dtp-OP-NA-ID",
                     'HI-IN_HI_dtp_NA_OP_ID_SP': "HI-IN-HI-dtp-OP-NA-ID-SP",
                     'hplt-v2-dedup': "All Registers (full HPLT v2)",#"HPLT v2",
                     'fineweb': "FineWeb",
                     'HI-IN': "Instructive-Informational (HI-IN)",
                     'HI': "How-to-Instructions (HI)",
                     'ID': "Interactive Discussion (ID)",
                     'IN': "Informational Description (IN)",
                     "IP": "Informational Persuasion (IP)",
                     "LY": "Lyrical (LY)",
                     "MT": "Machine Translation (MT)",
                     "NA": "Narrative (NA)",
                     "OP": "Opinion (OP)",
                     "SP": "Spoken (SP)",
                     "ne": "News (ne)",
                     "dtp": "Description (dtp)"}

benchmark_name_map = {'acc_norm_arc_easy':"ARC Easy",
                  'acc_norm_arc_challenge':"ARC Challenge",
                  'acc_norm_hellaswag': "Hellaswag",
                  'acc_norm_mmlu': "MMLU",
                  'acc_norm_siqa': "SIQA",
                  'acc_norm_piqa': "PIQA",
                  'acc_norm_winogrande': "WinoGrande",
                  'acc_norm_commonsense_qa': "CommonsenseQA",
                  'acc_norm_openbookqa': "OpenbookQA"}

random_baselines={'acc_norm_arc_easy':1/4.0,
                  'acc_norm_arc_challenge':1/4.0,
                  'acc_norm_hellaswag': 1/4.0,
                  'acc_norm_mmlu': 1/4.0,
                  'acc_norm_siqa': 1/3.0,
                  'acc_norm_piqa': 1/2.0,
                  'acc_norm_winogrande': 1/2.0,
                  'acc_norm_commonsense_qa': 1/5.0,
                  'acc_norm_openbookqa': 1/4.0
}

def create_colormap(baseline_cmap_name="Turbo"):


    c_map={}
    angles = np.linspace(0, 1, len(benchmark_name_map.keys())+1)# +1 because the map is cyclic
    color_scale = px.colors.sample_colorscale(baseline_cmap_name, angles)

    for i, bench in enumerate(benchmark_name_map.keys()):
        c_map[bench] = color_scale[i].replace("rgb", "rgba").replace(")", ", 1)")
    return c_map

def change_opacity(c_map, label, focus):
    if label in focus:
        return c_map[label]
    else:
        return c_map[label].replace(", 1)", ", 0.7)")

## Line plots, separated register

In [33]:


def plot_per_register(df, figname="plot"):
    global benchmark_name_map
    global register_name_map
    global vanilla_registers
    fig = make_subplots(
        rows=4, cols=3, 
        subplot_titles=[register_name_map[v] for v in vanilla_registers],
        vertical_spacing=0.05,  # Adjust space between subplots
        #horizontal_spacing=0.08
    )
    c_map = create_colormap()
    benchmarks = benchmark_name_map.keys()
    # Loop over each accuracy metric and add traces to the subplots
    for i, register in enumerate(vanilla_registers):
        # these are for baseline presentations
        add_baseline_rectangle=False
        x0, y0, x1, y1 = (0,1,0,1)

        col = int(i%3)+1
        row = int(i/3) + 1  
        label_data = df[df.register == register] 
        #print(f"{register} on row {row} column {col}")
        #continue
        for bm in benchmarks:
            dash_type = "solid" #if register in combinations else "solid"
            fig.add_trace(
                go.Scatter(
                    x=label_data['tokens'], 
                    y=label_data[bm], 
                    mode='lines', 
                    showlegend=True if row==1 and col==1 else False,
                    name=f'{benchmark_name_map.get(bm, bm)}', 
                    legendgroup=f'{bm}',
                    line=dict(color=c_map[bm], width=1.5, dash=dash_type)
                ),
                row=row, col=col
            )
            if label_data.min(axis=0)[bm] < random_baselines[bm]: #results are worse than baseline
                add_baseline_rectangle = True 
                x0 = label_data['tokens'].iloc[0] # these are static
                x1 = label_data['tokens'].iloc[-1] # does not matter if they are changed multiple times :)
                y0 = label_data.min(axis=0)[bm] if label_data.min(axis=0)[bm] < y0 else y0
                y1 = random_baselines[bm] # this is also static
                #print(f"Adding baseline thingy on {label} and {accuracy_col}, {label_data.min(axis=0)[accuracy_col]} < {random_baselines[accuracy_col]}")
                
        if False: #add_baseline_rectangle:
            fig.add_shape(
                type="rect",
                x0=x0, 
                x1=x1,
                y0=y0,
                y1=y1,
                fillcolor="gray",
                line_color="gray",
                opacity=0.2,
                row=row, col=col
                )
                
                

    # Update layout for better appearance
    fig.update_layout(
        title="",#"Accuracy Metrics by Task and Register",
        height=1300,  # Adjust the height of the plot
        width=900,
        font=dict(family="Computer Modern", size=16),
        legend_font_size=20,
        title_font_size=20,
        #title_font_size=40,
        showlegend=True,  # Show legend
        template="none",
        legend_tracegroupgap=15,
        margin=dict(l=50, r=50, t=20, b=20)
    )
    fig.update_annotations(font_size=20) #subplot titles are annotations
    #fig.write_image(f"sampolle/{figname}.png", scale=3)


    # Show the plot
    fig.show()

plot_per_register(df_rolled, figname="per_register")

In [34]:
means_per_task = df[df.step == "50000"]
means_per_task = means_per_task[means_per_task.register.isin(vanilla_registers)]
means = df.mean(axis=0)
#display(means_per_task)

means_per_task = {k: means[k] for k in benchmark_name_map.keys()}
print(means_per_task)

{'acc_norm_arc_easy': 0.4825817610062893, 'acc_norm_arc_challenge': 0.2630649895178197, 'acc_norm_hellaswag': 0.47973899371069173, 'acc_norm_mmlu': 0.28192972592565735, 'acc_norm_siqa': 0.39956498951781977, 'acc_norm_piqa': 0.7112389937106918, 'acc_norm_winogrande': 0.5314046121593292, 'acc_norm_commonsense_qa': 0.3332410901467506, 'acc_norm_openbookqa': 0.3152327044025158}


/tmp/ipykernel_193016/3144913799.py:3: FutureWarning:

The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



In [35]:



def plot_per_register_difference_to_mean(df, figname="plot"):
    global benchmark_name_map
    global register_name_map
    global vanilla_registers
    fig = make_subplots(
        rows=4, cols=3, 
        subplot_titles=[register_name_map[v] for v in vanilla_registers],
        vertical_spacing=0.05,  # Adjust space between subplots
        #horizontal_spacing=0.08
    )
    c_map = create_colormap()
    benchmarks = benchmark_name_map.keys()
    # Loop over each accuracy metric and add traces to the subplots
    for i, register in enumerate(vanilla_registers):
        # these are for baseline presentations
        add_baseline_rectangle=False
        x0, y0, x1, y1 = (0,1,0,1)

        col = int(i%3)+1
        row = int(i/3) + 1   
        label_data = df[df.register == register] 
        #print(f"{register} on row {row} column {col}")
        for bm in benchmarks:
            dash_type = "solid" if bm not in ["acc_norm_arc_challenge", "acc_norm_siqa", "acc_norm_commonsense_qa"] else "dot" #if register in combinations else "solid"
            fig.add_trace(
                go.Scatter(
                    x=label_data['tokens'], 
                    y=label_data[bm]-means_per_task[bm], 
                    mode='lines', 
                    showlegend=True if row==1 and col==1 else False,
                    name=f'{benchmark_name_map.get(bm, bm)}', 
                    legendgroup=f'{bm}',
                    line=dict(color=c_map[bm], width=1.5, dash=dash_type)
                ),
                row=row, col=col
            )
            
            if label_data.min(axis=0)[bm] < random_baselines[bm]: #results are worse than baseline
                add_baseline_rectangle = True 
                x0 = label_data['tokens'].iloc[0] # these are static
                x1 = label_data['tokens'].iloc[-1] # does not matter if they are changed multiple times :)
                y0 = label_data.min(axis=0)[bm] if label_data.min(axis=0)[bm] < y0 else y0
                y1 = random_baselines[bm] # this is also static
                #print(f"Adding baseline thingy on {label} and {accuracy_col}, {label_data.min(axis=0)[accuracy_col]} < {random_baselines[accuracy_col]}")
                

                
    #print(fig.layout)
    for i in range(1, 13):
        fig.update_layout({f'yaxis{i}': dict(range=[-0.25, 0.15])})           
    #print(fig.layout)
    # Update layout for better appearance
    fig.update_layout(
        title="",#"Accuracy Metrics by Task and Register",
        height=1300,  # Adjust the height of the plot
        width=1000,
        font=dict(family="Computer Modern", size=16),
        legend_font_size=20,
        title_font_size=20,
        #title_font_size=40,
        showlegend=True,  # Show legend
        template="none",
        legend_tracegroupgap=15,
        margin=dict(l=50, r=50, t=20, b=20)
    )
    fig.update_annotations(font_size=20) #subplot titles are annotations
    fig.write_image(f"by_register/{figname}.svg", scale=3)


    # Show the plot
    fig.show()

plot_per_register_difference_to_mean(df_rolled, figname="per_register")

## Line plots side by side

## Example for a Range plot

from https://stackoverflow.com/questions/63213527/is-there-a-way-to-create-range-plot-in-python-using-plotly

## Doing it for register data

## Latex table:



In [36]:
millnames = ['',' k',' M',' B',' T']
def millify(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))

    return '{:.0f}{}'.format(n / 10**(3 * millidx), millnames[millidx])


df_min_max = df[(df["step"] == "1000") | (df["step"] =="50000")]
to_latex= df_min_max[["register","step", "tokens", "acc_norm", "acc_norm_original", "acc_norm_stderr"]]

to_latex["register"] = to_latex["register"].map(lambda x: register_name_map.get(x,x))
to_latex["tokens"] = to_latex["tokens"].map(lambda x: millify(x))
to_latex = to_latex.sort_values(["register", "step"])
#display(to_latex)
#["register","step", "tokens", "acc_norm_original", "stderr_norm"]
print(to_latex.to_string(index=False))

                         register  step tokens  acc_norm  acc_norm_original  acc_norm_stderr
     All Registers (full HPLT v2)  1000    2 B  0.341536           0.265107         0.030274
     All Registers (full HPLT v2) 50000  105 B  0.457484           0.328140         0.032064
                Description (dtp)  1000    2 B  0.347940           0.268266         0.030463
                Description (dtp) 50000  105 B  0.452347           0.329816         0.032027
                          FineWeb  1000    2 B  0.348450           0.269707         0.030520
                          FineWeb 50000  105 B  0.465567           0.330767         0.031929
                     HI-IN-HI-dtp  1000    2 B  0.354339           0.265349         0.030291
                     HI-IN-HI-dtp 50000  105 B  0.465616           0.320815         0.031642
                  HI-IN-HI-dtp-OP  1000    2 B  0.352104           0.267529         0.030357
                  HI-IN-HI-dtp-OP 50000  105 B  0.475093           0.3

/tmp/ipykernel_193016/3739706883.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_193016/3739706883.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

